In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler

: 

In [ ]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1
from keras import backend as K
from keras.optimizers import Adam


In [ ]:


# Group LASSO functions
def l21_norm(W):
    return tf.reduce_sum(tf.norm(W, axis=1))

def group_regularization(weights, lambda_coeff=0.001):
    return lambda_coeff * l21_norm(weights)

def custom_loss(y_true, y_pred, weights, lambda_coeff=0.001):
    mse = tf.keras.losses.MeanSquaredError()(y_true, y_pred)
    return mse + group_regularization(weights, lambda_coeff)

def custom_loss_wrapper(lambda_coeff):
    def loss(y_true, y_pred):
        return custom_loss(y_true, y_pred, model.get_layer(index=1).kernel, lambda_coeff)
    return loss


# Step 1: Fetch the data.
tickers = ["META", "AAPL", "AMZN", "NFLX", "GOOGL", "GOLD", "VNQ", "QQQ"]
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=10)
data = yf.download(tickers, start=start_date, end=end_date)["Adj Close"]
returns = data.pct_change().dropna()
returns = returns[tickers]

# Prepare data.
LAGS = 2
X = []
y = []

for i in range(LAGS, len(returns) - 1):
    X.append(returns.iloc[i-LAGS:i].values.flatten())
    y.append(returns["QQQ"].iloc[i+1])

X = np.array(X)
y = np.array(y)

# Splitting data first
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data after splitting
scaler_X = MinMaxScaler().fit(X_train)
X_train_normalized = scaler_X.transform(X_train)
X_test_normalized = scaler_X.transform(X_test)

scaler_y = MinMaxScaler().fit(y_train.reshape(-1, 1))
y_train_normalized = scaler_y.transform(y_train.reshape(-1, 1)).ravel()
y_test_normalized = scaler_y.transform(y_test.reshape(-1, 1)).ravel()


returns.head()

In [6]:
# Model construction
input_layer = Input(shape=(X_train.shape[1],))
hidden_layer_1 = Dense(10, activation="tanh")(input_layer)
hidden_layer_2 = Dense(10, activation="tanh")(hidden_layer_1)
output_layer = Dense(1, activation="linear")(hidden_layer_2)
model = Model(inputs=input_layer, outputs=output_layer)

# Training with k-fold cross-validation
folds = 10
kf = KFold(n_splits=folds, shuffle=True, random_state=42)
best_mse = np.inf
best_lambda = None
lambdas = [1e-10, 1e-8, 1e-6, 1e-4, 1e-2]

for lam in lambdas:
    fold_mses = []
    for train, val in kf.split(X_train_normalized):
        model.compile(optimizer=Adam(), loss=custom_loss_wrapper(lam))
        model.fit(X_train_normalized[train], y_train_normalized[train], epochs=50, verbose=0)
        fold_mse = model.evaluate(X_train_normalized[val], y_train_normalized[val], verbose=0)
        fold_mses.append(fold_mse)

    avg_mse = np.mean(fold_mses)
    if avg_mse < best_mse:
        best_mse = avg_mse
        best_lambda = lam

# Train the model with the best lambda
model.compile(optimizer=Adam(), loss=custom_loss_wrapper(best_lambda))
model.fit(X_train_normalized, y_train_normalized, epochs=100, verbose=1)

Epoch 1/100
63/63 [==============================] - 1s 1ms/step - loss: 0.0062
Epoch 2/100
63/63 [==============================] - 0s 1ms/step - loss: 0.0061
Epoch 3/100
63/63 [==============================] - 0s 1ms/step - loss: 0.0063
Epoch 4/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 5/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 6/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 7/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 8/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 9/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 10/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 11/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 12/100
63/63 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 13/100
63/63 [=================

In [7]:
# Evaluation
predictions_normalized = model.predict(X_test_normalized)
predictions = scaler_y.inverse_transform(predictions_normalized)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"\nMSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R^2: {r2:.4f}")

# Output first-layer weights for Granger causality
weights_dict = {}
first_layer_weights = model.layers[1].get_weights()[0]

for i, ticker in enumerate(tickers[:-1]):
    weights_for_ticker = first_layer_weights[2*i : 2*(i+1)]
    weights_dict[ticker] = weights_for_ticker.mean(axis=1).tolist()

weights_df = pd.DataFrame.from_dict(weights_dict, orient='index').transpose()
weights_df.index = [f"Lag {i+1}" for i in range(LAGS)]
print(weights_df)
print(best_lambda)

16/16 [==============================] - 0s 1ms/step

MSE: 0.0002
MAE: 0.0100
R^2: -0.1058


           META      AAPL      AMZN      NFLX     GOOGL      GOLD       VNQ
Lag 1 -0.030227  0.012999 -0.056704 -0.005071  0.006862  0.007801 -0.014531
Lag 2 -0.072658  0.020765 -0.035618 -0.044817 -0.038121  0.055664 -0.017538
1e-08
